### import library

In [3]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly.express as px 

from sklearn.feature_extraction.text import TfidfVectorizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.metrics.pairwise import cosine_similarity

from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path

tv = TfidfVectorizer(max_features=5000)
stem = StemmerFactory().create_stemmer()
stopword = StopWordRemoverFactory().create_stop_word_remover()

In [4]:
import os
for dirname, _, filenames in os.walk('D:/Dataset Bangkit/archive/'): #membuka file
    for filename in filenames:
        print(os.path.join(dirname, filename))

D:/Dataset Bangkit/archive/package_tourism.csv
D:/Dataset Bangkit/archive/tourism_rating.csv
D:/Dataset Bangkit/archive/tourism_with_id.csv
D:/Dataset Bangkit/archive/user.csv


### data understanding

In [5]:
turis_pack = pd.read_csv('D:/Dataset Bangkit/archive/package_tourism.csv')
rating = pd.read_csv('D:/Dataset Bangkit/archive/tourism_rating.csv')
turis_id = pd.read_csv('D:/Dataset Bangkit/archive/tourism_with_id.csv')
user = pd.read_csv('D:/Dataset Bangkit/archive/user.csv')

In [6]:
turis_pack.head()

Package     City         Place_Tourism1      Place_Tourism2  \
0        1  Jakarta      Pasar Tanah Abang        Taman Ayodya   
1        2  Jakarta      Pasar Tanah Abang  Pasar Taman Puring   
2        3  Jakarta  Perpustakaan Nasional               Monas   
3        4  Jakarta           Pulau Tidung      Pulau Bidadari   
4        5  Jakarta  Museum Satria Mandala       Museum Wayang   

          Place_Tourism3                                     Place_Tourism4  \
0         Museum Tekstil                                                NaN   
1   Pasar Petak Sembilan                                                NaN   
2        Masjid Istiqlal                                                NaN   
3             Pulau Pari                                      Pulau Pramuka   
4  Museum Bahari Jakarta  Museum Macan (Modern and Contemporary Art in N...   

  Place_Tourism5  
0            NaN  
1            NaN  
2            NaN  
3  Pulau Pelangi  
4            NaN

In [7]:
rating.head()

User_Id  Place_Id  Place_Ratings
0        1       179              3
1        1       344              2
2        1         5              5
3        1       373              3
4        1       101              4

In [8]:
turis_id.head()

Place_Id                         Place_Name  \
0         1                   Monumen Nasional   
1         2                           Kota Tua   
2         3                      Dunia Fantasi   
3         4  Taman Mini Indonesia Indah (TMII)   
4         5           Atlantis Water Adventure   

                                         Description       Category     City  \
0  Monumen Nasional atau yang populer disingkat d...         Budaya  Jakarta   
1  Kota tua di Jakarta, yang juga bernama Kota Tu...         Budaya  Jakarta   
2  Dunia Fantasi atau disebut juga Dufan adalah t...  Taman Hiburan  Jakarta   
3  Taman Mini Indonesia Indah merupakan suatu kaw...  Taman Hiburan  Jakarta   
4  Atlantis Water Adventure atau dikenal dengan A...  Taman Hiburan  Jakarta   

    Price  Rating  Time_Minutes  \
0   20000     4.6          15.0   
1       0     4.6          90.0   
2  270000     4.6         360.0   
3   10000     4.5           NaN   
4   94000     4.5          60.0   

                                        Coordinate       Lat        Long  \
0          {'lat': -6.1753924, 'lng': 106.8271528} -6.175392  106.827153   
1  {'lat': -6.137644799999999, 'lng': 106.8171245} -6.137645  106.817125   
2  {'lat': -6.125312399999999, 'lng': 106.8335377} -6.125312  106.833538   
3  {'lat': -6.302445899999999, 'lng': 106.8951559} -6.302446  106.895156   
4             {'lat': -6.12419, 'lng': 106.839134} -6.124190  106.839134   

   Unnamed: 11  Unnamed: 12  
0          NaN            1  
1          NaN            2  
2          NaN            3  
3          NaN            4  
4          NaN            5

In [9]:
user.head()

User_Id                   Location  Age
0        1      Semarang, Jawa Tengah   20
1        2         Bekasi, Jawa Barat   21
2        3        Cirebon, Jawa Barat   23
3        4         Bekasi, Jawa Barat   21
4        5  Lampung, Sumatera Selatan   20

In [11]:
print(f"Number of places in the datasets : {len(turis_id.Place_Id.unique())}")
print(f"Number of users : {len(user.User_Id.unique())}")
print(f"The number of ratings given by the user to the dataset : {len(rating.User_Id)}")

Number of places in the datasets : 437
Number of users : 300
The number of ratings given by the user to the dataset : 10000


In [12]:
rating.groupby('Place_Id').agg(
    jumlah_pengunjung = ('User_Id', 'count')).reset_index()

Place_Id  jumlah_pengunjung
0           1                 18
1           2                 25
2           3                 19
3           4                 21
4           5                 25
..        ...                ...
432       433                 23
433       434                 25
434       435                 17
435       436                 22
436       437                 38

[437 rows x 2 columns]

### EDA

In [13]:
turis_id.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 437 entries, 0 to 436
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Place_Id      437 non-null    int64  
 1   Place_Name    437 non-null    object 
 2   Description   437 non-null    object 
 3   Category      437 non-null    object 
 4   City          437 non-null    object 
 5   Price         437 non-null    int64  
 6   Rating        437 non-null    float64
 7   Time_Minutes  205 non-null    float64
 8   Coordinate    437 non-null    object 
 9   Lat           437 non-null    float64
 10  Long          437 non-null    float64
 11  Unnamed: 11   0 non-null      float64
 12  Unnamed: 12   437 non-null    int64  
dtypes: float64(5), int64(3), object(5)
memory usage: 44.5+ KB


In [14]:
turis_id.isnull().sum() # mencari missing value

Place_Id          0
Place_Name        0
Description       0
Category          0
City              0
Price             0
Rating            0
Time_Minutes    232
Coordinate        0
Lat               0
Long              0
Unnamed: 11     437
Unnamed: 12       0
dtype: int64

In [15]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   User_Id        10000 non-null  int64
 1   Place_Id       10000 non-null  int64
 2   Place_Ratings  10000 non-null  int64
dtypes: int64(3)
memory usage: 234.5 KB


In [16]:
rating.isnull().sum()

User_Id          0
Place_Id         0
Place_Ratings    0
dtype: int64

In [17]:
turis_id.Category.unique()

array(['Budaya', 'Taman Hiburan', 'Cagar Alam', 'Bahari',
       'Pusat Perbelanjaan', 'Tempat Ibadah'], dtype=object)

### data prepocessing

In [18]:
all_turis_rate = rating
all_turis_rate

User_Id  Place_Id  Place_Ratings
0           1       179              3
1           1       344              2
2           1         5              5
3           1       373              3
4           1       101              4
...       ...       ...            ...
9995      300       425              2
9996      300        64              4
9997      300       311              3
9998      300       279              4
9999      300       163              2

[10000 rows x 3 columns]

In [19]:
turis_id.drop(['Rating','Time_Minutes','Coordinate','Lat','Long','Unnamed: 11','Unnamed: 12'],axis=1,inplace=True)
turis_id

Place_Id                                 Place_Name  \
0           1                           Monumen Nasional   
1           2                                   Kota Tua   
2           3                              Dunia Fantasi   
3           4          Taman Mini Indonesia Indah (TMII)   
4           5                   Atlantis Water Adventure   
..        ...                                        ...   
432       433                        Museum Mpu Tantular   
433       434                              Taman Bungkul   
434       435           Taman Air Mancur Menari Kenjeran   
435       436               Taman Flora Bratang Surabaya   
436       437  Gereja Perawan Maria Tak Berdosa Surabaya   

                                           Description       Category  \
0    Monumen Nasional atau yang populer disingkat d...         Budaya   
1    Kota tua di Jakarta, yang juga bernama Kota Tu...         Budaya   
2    Dunia Fantasi atau disebut juga Dufan adalah t...  Taman Hiburan   
3    Taman Mini Indonesia Indah merupakan suatu kaw...  Taman Hiburan   
4    Atlantis Water Adventure atau dikenal dengan A...  Taman Hiburan   
..                                                 ...            ...   
432  Museum Negeri Mpu Tantular adalah sebuah museu...         Budaya   
433  Taman Bungkul adalah taman wisata kota yang te...  Taman Hiburan   
434  Air mancur menari atau dancing fountain juga a...  Taman Hiburan   
435  Taman Flora adalah salah satu taman kota di Su...  Taman Hiburan   
436  Gereja Katolik Kelahiran Santa Perawan Maria m...  Tempat Ibadah   

         City   Price  
0     Jakarta   20000  
1     Jakarta       0  
2     Jakarta  270000  
3     Jakarta   10000  
4     Jakarta   94000  
..        ...     ...  
432  Surabaya    2000  
433  Surabaya       0  
434  Surabaya       0  
435  Surabaya       0  
436  Surabaya   10000  

[437 rows x 6 columns]

In [20]:
all_tourism = pd.merge(rating.groupby('Place_Id')['Place_Ratings'].mean(),turis_id,on='Place_Id')
all_tourism # melakukan merge kolom yang dibutuhkan untuk model

Place_Id  Place_Ratings                                 Place_Name  \
0           1       3.722222                           Monumen Nasional   
1           2       2.840000                                   Kota Tua   
2           3       2.526316                              Dunia Fantasi   
3           4       2.857143          Taman Mini Indonesia Indah (TMII)   
4           5       3.520000                   Atlantis Water Adventure   
..        ...            ...                                        ...   
432       433       3.304348                        Museum Mpu Tantular   
433       434       2.400000                              Taman Bungkul   
434       435       3.000000           Taman Air Mancur Menari Kenjeran   
435       436       3.090909               Taman Flora Bratang Surabaya   
436       437       3.289474  Gereja Perawan Maria Tak Berdosa Surabaya   

                                           Description       Category  \
0    Monumen Nasional atau yang populer disingkat d...         Budaya   
1    Kota tua di Jakarta, yang juga bernama Kota Tu...         Budaya   
2    Dunia Fantasi atau disebut juga Dufan adalah t...  Taman Hiburan   
3    Taman Mini Indonesia Indah merupakan suatu kaw...  Taman Hiburan   
4    Atlantis Water Adventure atau dikenal dengan A...  Taman Hiburan   
..                                                 ...            ...   
432  Museum Negeri Mpu Tantular adalah sebuah museu...         Budaya   
433  Taman Bungkul adalah taman wisata kota yang te...  Taman Hiburan   
434  Air mancur menari atau dancing fountain juga a...  Taman Hiburan   
435  Taman Flora adalah salah satu taman kota di Su...  Taman Hiburan   
436  Gereja Katolik Kelahiran Santa Perawan Maria m...  Tempat Ibadah   

         City   Price  
0     Jakarta   20000  
1     Jakarta       0  
2     Jakarta  270000  
3     Jakarta   10000  
4     Jakarta   94000  
..        ...     ...  
432  Surabaya    2000  
433  Surabaya       0  
434  Surabaya       0  
435  Surabaya       0  
436  Surabaya   10000  

[437 rows x 7 columns]

In [21]:
all_tourism

Place_Id  Place_Ratings                                 Place_Name  \
0           1       3.722222                           Monumen Nasional   
1           2       2.840000                                   Kota Tua   
2           3       2.526316                              Dunia Fantasi   
3           4       2.857143          Taman Mini Indonesia Indah (TMII)   
4           5       3.520000                   Atlantis Water Adventure   
..        ...            ...                                        ...   
432       433       3.304348                        Museum Mpu Tantular   
433       434       2.400000                              Taman Bungkul   
434       435       3.000000           Taman Air Mancur Menari Kenjeran   
435       436       3.090909               Taman Flora Bratang Surabaya   
436       437       3.289474  Gereja Perawan Maria Tak Berdosa Surabaya   

                                           Description       Category  \
0    Monumen Nasional atau yang populer disingkat d...         Budaya   
1    Kota tua di Jakarta, yang juga bernama Kota Tu...         Budaya   
2    Dunia Fantasi atau disebut juga Dufan adalah t...  Taman Hiburan   
3    Taman Mini Indonesia Indah merupakan suatu kaw...  Taman Hiburan   
4    Atlantis Water Adventure atau dikenal dengan A...  Taman Hiburan   
..                                                 ...            ...   
432  Museum Negeri Mpu Tantular adalah sebuah museu...         Budaya   
433  Taman Bungkul adalah taman wisata kota yang te...  Taman Hiburan   
434  Air mancur menari atau dancing fountain juga a...  Taman Hiburan   
435  Taman Flora adalah salah satu taman kota di Su...  Taman Hiburan   
436  Gereja Katolik Kelahiran Santa Perawan Maria m...  Tempat Ibadah   

         City   Price  
0     Jakarta   20000  
1     Jakarta       0  
2     Jakarta  270000  
3     Jakarta   10000  
4     Jakarta   94000  
..        ...     ...  
432  Surabaya    2000  
433  Surabaya       0  
434  Surabaya       0  
435  Surabaya       0  
436  Surabaya   10000  

[437 rows x 7 columns]

In [22]:
all_tourism.isnull().sum()

Place_Id         0
Place_Ratings    0
Place_Name       0
Description      0
Category         0
City             0
Price            0
dtype: int64

In [23]:
all_tourism.to_csv('tempat_wisata.csv') #memisahkan data ke file csv baru